In [989]:
%pip install torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import ReduceLROnPlateau

Note: you may need to restart the kernel to use updated packages.


In [990]:
# ワインデータセットの読み込み
wine = load_wine()
x = wine['data']
t = wine['target']

In [991]:
# データの標準化 
scaler = StandardScaler() 
x = scaler.fit_transform(x) 
x = torch.tensor(x, dtype=torch.float32) 
t = torch.tensor(t, dtype=torch.int64)

In [992]:
# データセットの作成
dataset = torch.utils.data.TensorDataset(x,t)

In [993]:
# データの分割 
n_train = int(len(dataset) * 0.6) 
n_val = int(len(dataset) * 0.2) 
n_test = len(dataset) - n_train - n_val

In [994]:
# 乱数シードを設定 
torch.manual_seed(0) 
# データセットの分割
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [995]:
# DataLoaderの作成(バッチサイズ)train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False) 
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [996]:
# ネットワークの定義 13-64-32-16-3（ワインデータセットは13個の特徴量） 
class Net(nn.Module): 
    def __init__(self): 
        super().__init__() 
        self.fc1 = nn.Linear(13, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.fc5 = nn.Linear(32, 16)
        self.bn5 = nn.BatchNorm1d(16)
        self.fc6 = nn.Linear(16, 3)
        self.dropout = nn.Dropout(0.5) # 50%のドロップアウト率
        
    def forward(self, x): 
        x = F.relu(self.fc1(x)) 
        x = self.dropout(x)
        x = F.relu(self.fc2(x)) 
        x = self.dropout(x)
        x = F.relu(self.fc3(x)) 
        x = self.dropout(x) 
        x = F.relu(self.fc4(x)) 
        x = self.dropout(x) 
        x = F.relu(self.fc5(x)) 
        x = self.fc6(x) 
        return x

In [997]:
# 重みの初期化 
def weights_init(m): 
    if isinstance(m, nn.Linear): 
        nn.init.xavier_normal_(m.weight) 
        nn.init.zeros_(m.bias)

In [998]:
# デバイスの設定 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [999]:
# モデルのインスタンス化と最適化アルゴリズムの設定 
torch.manual_seed(0) 
net = Net().to(device) 
net.apply(weights_init) 
optimizer = torch.optim.SGD(net.parameters(), lr=0.0001, momentum=0.9, weight_decay=1e-4)

In [1011]:
# 学習率スケジューラの設定 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
# エポック数の設定 
max_epoch = 100

In [1039]:
# 変数の初期化 
best_val_accuracy = 0 
best_model_state = None 
patience_counter = 0 
patience_threshold = 10 # 早期停止の閾値

for epoch in range(max_epoch): 
    net.train() 
    for batch in train_loader: 
        x, t = batch 
        x = x.to(device) 
        t = t.to(device) 
        y = net(x) 
        loss = F.cross_entropy(y, t) 
        
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 

# 学習率のステップを更新
    net.eval() 
    val_loss = 0
    with torch.no_grad(): 
        total_correct = 0 
        for batch in val_loader: 
            x, t = batch 
            x = x.to(device) 
            t = t.to(device) 
            y = net(x) 
            y_label = torch.argmax(y, dim=1) 
            val_loss += F.cross_entropy(y, t, reduction='sum').item() 
            y_label = torch.argmax(y, dim=1)
            total_correct += (y_label == t).sum().item() 
        val_accuracy = total_correct / len(val_loader.dataset) 
        val_loss /= len(val_loader.dataset)
        scheduler.step(val_loss)
    print(f'Epoch {epoch + 1}/{max_epoch}, Validation Accuracy: {val_accuracy:.2f}') 
    # 早期停止のチェック 
    if val_accuracy > best_val_accuracy: 
        best_val_accuracy = val_accuracy 
        best_model_state = net.state_dict() 
        patience_counter = 0 
    else: 
        patience_counter += 1 
        if patience_counter >= patience_threshold: 
            print("Early stopping due to no improvement in validation accuracy") 
            break


Epoch 1/100, Validation Accuracy: 0.63
Epoch 2/100, Validation Accuracy: 0.60
Epoch 3/100, Validation Accuracy: 0.60
Epoch 4/100, Validation Accuracy: 0.71
Epoch 5/100, Validation Accuracy: 0.74
Epoch 6/100, Validation Accuracy: 0.91
Epoch 7/100, Validation Accuracy: 0.89
Epoch 8/100, Validation Accuracy: 0.97
Epoch 9/100, Validation Accuracy: 0.97
Epoch 10/100, Validation Accuracy: 0.97
Epoch 11/100, Validation Accuracy: 0.94
Epoch 12/100, Validation Accuracy: 0.97
Epoch 13/100, Validation Accuracy: 0.94
Epoch 14/100, Validation Accuracy: 0.86
Epoch 15/100, Validation Accuracy: 0.83
Epoch 16/100, Validation Accuracy: 0.89
Epoch 17/100, Validation Accuracy: 0.91
Epoch 18/100, Validation Accuracy: 0.91
Early stopping due to no improvement in validation accuracy


In [1045]:
# ベストモデルのロード 
if best_model_state is not None: 
    net.load_state_dict(best_model_state) 
else: 
    print("No best model state found. Training did not improve validation accuracy.")

In [1047]:
# 最終テストデータでの評価 
net.eval() 
with torch.no_grad(): 
    total_correct = 0 
    for batch in test_loader: 
        x, t = batch 
        x = x.to(device) 
        t = t.to(device) 
        y = net(x) 
        y_label = torch.argmax(y, dim=1) 
        total_correct += (y_label == t).sum().item() 
    final_accuracy = total_correct / len(test_loader.dataset) 
    print(f'Final Test Accuracy: {final_accuracy:.2f}')

Final Test Accuracy: 0.92


In [1049]:
#検証データの正解率
calc_accuracy(val_loader)

tensor(0.9143)

In [1051]:
#テストデータの正解率
calc_accuracy(test_loader)

tensor(0.9189)